# European Soccer Data Analysis
### Data Engineering Capstone Project

#### Drafting and Data Exploring *Don't Run Again*

In this section of the project I was some sort of drafting and getting my ideas right to start building the pipeline. I made some transformations and cleansing on the provided sqlite database. I committed the changes and the uploaded version of the database is the new one after cleansing. I will document here some of my thinking process.


In [3]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import sqlite3
import re
import requests
import psycopg2
from datetime import datetime, date
from bs4 import BeautifulSoup as Soup

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
# Defining dataframe columns to start scraping
column = ['ID','Name','At_the_time_Age','Overall_Rating','Team','Sub','Wage','Total_Stats','Date']
df__scraped_players = pd.DataFrame(columns = column)

In [ ]:
# Scraping players
for offset in range(80002,170002,10000):
    if offset in [80002,90002]:
        url = (f"https://sofifa.com/players?type=all&ct%5B0%5D=2&r=0{offset}&set=true")
    else:
        url = (f"https://sofifa.com/players?type=all&ct%5B0%5D=2&r={offset}&set=true")
    for j in range (0,30):
        url = url + (f"&offset={j*60}")
        p_html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        p_soup = p_html.text
        data = Soup(p_soup,'html.parser')
        table = data.find('tbody')
        for i in table.findAll('tr'):   
            td = i.findAll('td')
            ID = td[0].find('img').get('id')
            Name = td[1].findAll('a')[0].text
            Age = td[2].text.split()
            Overall_Rating = td[3].find('span').text
            Team = td[5].find('a').text
            Sub = td[5].find('div',{'class':'sub'}).text.strip()
            Wage = td[7].text.strip()
            Total_Stats = td[8].text.strip()
            Date = data.findAll('span',{'class':'bp3-button-text'})[1].text
            player_data = pd.DataFrame([[ID,Name,Age,Overall_Rating,Team,Sub,Wage,Total_Stats,Date]])
            player_data.columns = column
            df__scraped_players = df__scraped_players.append(player_data, ignore_index = True)

In [ ]:
df__scraped_players.head()

,ID,Name,At_the_time_Age,Overall_Rating,Team,Sub,Wage,Total_Stats,Date
0,20801,Cristiano Ronaldo,22,91,Manchester United,2012,0,1715,2008-02-22
1,1625,T. Henry,29,91,FC Barcelona,2011,0,1696,2008-02-22
2,1179,G. Buffon,29,91,Juventus,2012,0,1276,2008-02-22
3,1088,A. Nesta,31,91,Milan,2011,0,1523,2008-02-22
4,177176,H. de Noteboom,31,90,111648,,0,1713,2008-02-22


In [ ]:
# Converting date column to datetime format
df__scraped_players['Date'] = pd.to_datetime(df__scraped_players['Date'])

In [9]:
# Defining dataframe columns to start scraping
column = ['ID','Team_Name']
df_scraped_teams_data = pd.DataFrame(columns = column)

In [10]:
# Scraping teams
for j in range (0,9):
    url = (f"https://sofifa.com/teams?type=all&ct%5B%5D=2&offset={j*60}")
    p_html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    p_soup = p_html.text
    data = Soup(p_soup,'html.parser')
    table = data.find('tbody')
    for i in table.findAll('tr'):   
        td = i.findAll('td')
        x = td[0].find('img').get('data-src')
        x = re.split('https://cdn.sofifa.net/teams/|/60.png',x)
        ID = x[1]
        Team_Name = td[1].findAll('a')[0].text
        team_data = pd.DataFrame([[ID,Team_Name]])
        team_data.columns = column
        df_scraped_teams_data = df_scraped_teams_data.append(team_data, ignore_index = True)

In [5]:
df = pd.read_csv('scraped_teams.csv')

In [7]:
# Checking data types

df.dtypes

Unnamed: 0     int64
ID             int64
Team_Name     object
dtype: object

In [11]:
df_scraped_teams_data.dtypes

ID           object
Team_Name    object
dtype: object

In [3]:
# Importing database

database = "database.sqlite"

conn = sqlite3.connect(database)
cur = conn.cursor()
   

 #extract list of countries
df_country = pd.read_sql_query('Select * from Country', conn)
df_country
# a list of countries:

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [4]:
df_country.to_csv('country_exported.csv')

In [85]:
# Reading teams data

df_team = pd.read_sql_query('Select * from Team', conn)

In [49]:
df_team

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [52]:
# Deleting duplicate id 

cur.execute('delete from Team where team_api_id = 8024')

In [58]:

#  Deleting duplicate id
cur.execute('update Team_Attributes set team_api_id = 8244 where team_api_id = 8024')

In [147]:
df_player = pd.read_sql_query('Select * from Player', conn)
df_player

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,31,182.88,187
1,2,155782,Aaron Cresswell,189615,34,170.18,146
2,3,162549,Aaron Doran,186170,32,170.18,163
3,4,30572,Aaron Galindo,140161,41,182.88,198
4,5,23780,Aaron Hughes,17725,44,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,44,182.88,168
11056,11072,111182,Zsolt Laczko,164680,37,182.88,176
11057,11073,36491,Zsolt Low,111191,44,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,42,185.42,172


In [19]:
# I converted the birthday column to his corresponding age now
cur.execute("update Player set birthday = date()-birthday ")

In [ ]:
# Renaaming column
df_player.rename(columns={'birthday':'age'},inplace=True)
df_player

In [ ]:
data.to_sql('public.dim_player', con = conn, if_exists = 'append', chunksize = 1000)

In [220]:
df_match = pd.read_sql_query('Select * from Match', conn)

In [30]:
df_match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_league = pd.read_sql_query('Select * from League', conn)
df_league

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [5]:
df_team_attr =  pd.read_sql_query('Select * from Team_Attributes', conn)
df_team_attr.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [6]:
df_player_attr = pd.read_sql_query('Select * from Player_Attributes', conn)
df_player_attr.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [11]:
# Start date parsing
dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [12]:
df_time = pd.read_sql('select distinct date from Match', conn)
df_time

,date
0,2008-08-17 00:00:00
1,2008-08-16 00:00:00
2,2008-09-24 00:00:00
3,2008-11-01 00:00:00
4,2008-10-31 00:00:00
...,...
1689,2014-08-14 00:00:00
1690,2016-05-16 00:00:00
1691,2016-05-22 00:00:00
1692,2016-05-25 00:00:00


In [72]:
df_team.dtypes

id                    int64
team_api_id           int64
team_fifa_api_id    float64
team_long_name       object
team_short_name      object
dtype: object

In [39]:
df.values

array([[0, 10.0, 'Manchester City'],
       [1, 21.0, 'FC Bayern München'],
       [2, 1337.0, 'Germany'],
       ...,
       [500, 111131.0, 'Finn Harps'],
       [501, 1572.0, 'Drogheda United'],
       [502, 111132.0, 'UCD']], dtype=object)

In [138]:
df_team = pd.read_sql_query('Select * from Team', conn)

In [136]:
cur.execute('delete from Team where team_api_id = 8020')

In [134]:
cur.execute('update Team_Attributes set team_api_id = 8031 where team_api_id = 8020')

In [144]:
conn.commit()